In [1]:
import os 
import numpy as np 
import json

In [2]:
def read_report(expr_dir):
    test_mode=['val', 'test']
    data = {}
    for mode in test_mode:
        report = os.path.join(expr_dir, 'report-trained.val-tested.{}.json'.format(mode))
        with open(report, 'r') as f:
            data[mode] = json.load(f)
    return data

# BETAS

In [69]:
# Read SFNN
run_dir = '../../pib_run/train'
particles='32,32'
gamma1='1.0'
gamma2='1.0'
beta1='0.0001'
beta2='0.0001'

sto_val = np.zeros((30,))
sto_test = np.zeros((30,))
det_val = np.zeros((30,))
det_test =np.zeros((30,))
metric_val = np.zeros((30,))
for i in range(30):
    run = str(i+1)
    model_name="PIB.512x2.adamax.0.01.g1.{}-g2.{}-beta1.{}-beta2.{}-run.{}".format(gamma1, gamma2, beta1, beta2, run)
    data = read_report(os.path.join(run_dir, model_name), particles)
    sto_val[i] = data['val']['pib_test_err_mean']
    sto_test[i] = data['test']['pib_test_err_mean']
    det_val[i] = data['val']['det_test_err']
    det_test[i] = data['test']['det_test_err']
    metric_val[i] = data['val']['stochastic_metric']
    
print(np.mean(sto_test), np.var(sto_test))
print(np.mean(det_test), np.var(det_test))
print(np.mean(sto_val), np.var(sto_val))
print(np.mean(det_val), np.var(det_val))

1.4304633333333334 0.0006441556555555592
1.421999999999999 0.0006093333333332924
1.1689666666666667 0.0023402322222222318
1.1633333333333347 0.0036555555555555625


# VCR 

In [102]:
# Read VCR
NUM_RUNS=15
run_dir = '../../pib_run/train'
particles='32,32'

gammas1 = ['0.0', '0.2', '0.4', '0.6', '0.8', '1.0']
gammas2 = ['0.0', '0.2', '0.4', '0.6', '0.8', '1.0']

hyperparam1 = []
hyperparam2 = []
hyperparam = []
sto_test_mean = []
sto_test_var = []
det_test_mean = []
det_test_var = []
sto_val_mean = []
sto_val_var = []
det_val_mean = []
det_val_var = []

for gamma1 in gammas1:
    for gamma2 in gammas2:
        hyperparam1.append(gamma1)
        hyperparam2.append(gamma2)
        hyperparam.append('gamma1.{},gamma2.{}'.format(gamma1, gamma2))
        sto_val = np.zeros((NUM_RUNS,))
        sto_test = np.zeros((NUM_RUNS,))
        det_val = np.zeros((NUM_RUNS,))
        det_test =np.zeros((NUM_RUNS,))
        metric_val = np.zeros((NUM_RUNS,))
        for i in range(NUM_RUNS):
            run = str(i+1)
            model_name="VCR.512x2.adamax.0.01.g1.{}-g2.{}-run.{}".format(gamma1, gamma2, run)
            data = read_report(os.path.join(run_dir, model_name), particles)
            sto_val[i] = data['val']['pib_test_err_mean']
            sto_test[i] = data['test']['pib_test_err_mean']
            det_val[i] = data['val']['det_test_err']
            det_test[i] = data['test']['det_test_err']
            metric_val[i] = data['val']['stochastic_metric']
        
        sto_test_mean.append(np.mean(sto_test))
        sto_test_var.append(np.var(sto_test))
        det_test_mean.append(np.mean(det_test))
        det_test_var.append(np.var(det_test))
        sto_val_mean.append(np.mean(sto_val))
        sto_val_var.append(np.var(sto_val))
        det_val_mean.append(np.mean(det_val))
        det_val_var.append(np.var(det_val))
        
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
 
df = pd.DataFrame({'gamma': hyperparam, 
                   'gamma1': hyperparam1,
                   'gamma2': hyperparam2, 
                   'sto_test_mean': sto_test_mean, 
                   'sto_test_var': sto_test_var, 
                   'det_test_mean': det_test_mean, 
                   'det_test_var': det_test_var, 
                   'sto_val_mean': sto_val_mean, 
                   'sto_val_var': sto_val_var, 
                   'det_val_mean': det_val_mean, 
                   'det_val_var': det_val_var})
 
writer = ExcelWriter('VCR_{}runs.xlsx'.format(NUM_RUNS))
df.to_excel(writer,'VCR.512x2.adamax.0.01',index=False)
writer.save()

# PIB

In [98]:
gammas = ['0.0', '0.2', '0.4', '0.6', '0.8', '1.0']
betas = ['1.0', '0.1', '0.01', '0.001', '0.0001', '0.00001']

hyperparam1 = []
hyperparam2 = []
hyperparam = []
sto_test_mean = []
sto_test_var = []
det_test_mean = []
det_test_var = []
sto_val_mean = []
sto_val_var = []
det_val_mean = []
det_val_var = []

for gamma in gammas:
    for beta in betas:
        hyperparam1.append(gamma)
        hyperparam2.append(beta)
        hyperparam.append('gamma.{},beta.{}'.format(gamma, beta))
        sto_val = np.zeros((30,))
        sto_test = np.zeros((30,))
        det_val = np.zeros((30,))
        det_test =np.zeros((30,))
        metric_val = np.zeros((30,))
        for i in range(30):
            run = str(i+1)
            model_name="PIB.512x2.adamax.0.01.g1.{}-g2.{}-beta1.{}-beta2.{}-run.{}".format(gamma, gamma,beta,beta, run)
            data = read_report(os.path.join(run_dir, model_name), particles)
            sto_val[i] = data['val']['pib_test_err_mean']
            sto_test[i] = data['test']['pib_test_err_mean']
            det_val[i] = data['val']['det_test_err']
            det_test[i] = data['test']['det_test_err']
            metric_val[i] = data['val']['stochastic_metric']
        
        sto_test_mean.append(np.mean(sto_test))
        sto_test_var.append(np.var(sto_test))
        det_test_mean.append(np.mean(det_test))
        det_test_var.append(np.var(det_test))
        sto_val_mean.append(np.mean(sto_val))
        sto_val_var.append(np.var(sto_val))
        det_val_mean.append(np.mean(det_val))
        det_val_var.append(np.var(det_val))
        
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
 
df = pd.DataFrame({ 'gamma_beta': hyperparam, 
                   'gamma': hyperparam1,
                   'beta': hyperparam2, 
                   'sto_test_mean': sto_test_mean, 
                   'sto_test_var': sto_test_var, 
                   'det_test_mean': det_test_mean, 
                   'det_test_var': det_test_var, 
                   'sto_val_mean': sto_val_mean, 
                   'sto_val_var': sto_val_var, 
                   'det_val_mean': det_val_mean, 
                   'det_val_var': det_val_var})
 
writer = ExcelWriter('PIB.xlsx')
df.to_excel(writer,'PIB.512x2.adamax.0.01',index=False)
writer.save()